In [9]:
# Australian news sites

import csv
import requests
from bs4 import BeautifulSoup
import anthropic
import re


import API_KEYS

In [10]:
GUARDIAN_API_KEY = API_KEYS.GUARDIAN_API_KEY
ANTHROPIC_API_KEY = API_KEYS.ANTHROPIC_API_KEY

In [11]:
url = f"https://content.guardianapis.com/search?section=australia-news&api-key={GUARDIAN_API_KEY}"
response = requests.get(url)


# Check if the request was successful
if response.status_code == 200:

    data = response.json()

    results = data["response"]["results"]
else:
    print(f"Failed to fetch data: {response.status_code}")

In [ ]:
client = anthropic.Anthropic(
    api_key=ANTHROPIC_API_KEY
)

In [ ]:
def extract_title_and_content(input_text):
    # Extract title using regular expression
    title_match = re.search(r'title:\{(.*?)\}', input_text, re.DOTALL)
    title = title_match.group(1).strip() if title_match else "Title not found"

    # Extract content using regular expression
    content_match = re.search(r'content:\{(.*?)\}', input_text, re.DOTALL)
    content = content_match.group(1).strip() if content_match else "Content not found"

    return title, content

In [48]:
def summarise_news_article(title, content) -> list[str, str]:
    system_prompt = """

    You are a expert journalist, who writes in a beautiful narrative style.
    Given a news title and content you are to write your own story using that information.

    title:{title}

    content:{content}

    """


    user_prompt = f"""

    Write your own story based on this news story

    title:{title}

    content:{content}

    Return your story in this exact format, Surrond the title and Content with Curly Braces

    title:{"Story Here"}
    content:{"Content Here"}


    """

    message = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1000,
        temperature=0,
        system=system_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_prompt
                    }
                ]
            },
        ]
    )

    response = message.content

    AI_title, AI_content = extract_title_and_content(response[0].text)
    return AI_title, AI_content


In [49]:
with open("guardian_news_results.csv" ,"w") as file:
    writer = csv.writer(file)

    writer.writerow(["title", "date", "url", "full_text","ai_title","ai_content"])

    for result in results:

        title = result["webTitle"]
        date = result["webPublicationDate"]
        date = date[:10] #trim off extra time information
        url = result["webUrl"]

        # Scrape the text of the article
        headers = {'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"}
        response = requests.get(url, headers=headers)
        page_soup = BeautifulSoup(response.content, features="html.parser")


        full_text = ""

        for x in page_soup.find_all('p'):
            full_text = full_text + x.get_text() + "\n"


        ai_title, ai_content = summarise_news_article(title, full_text)


        writer.writerow([
            title,
            date,
            url,
            full_text,
            ai_title,
            ai_content,
        ])

print("Scraping Complete")

Scraping Complete
